In [3]:
import socket
import threading
import json
import time

# Server Configuration
SERVER_IP = "192.168.233.79"
BROADCAST_PORT = 59073
TCP_PORT = 59074
HEARTBEAT_INTERVAL = 5

# List of connected peers
peers = []


def debug(message):
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(f"[{current_time}] {message}")


def encode_message(message):
    return json.dumps(message).encode('utf-8')


def decode_message(message):
    return json.loads(message.decode('utf-8'))


class BroadcastSender(threading.Thread):
    def _init_(self):
        threading.Thread._init_(self)

    def run(self):
        broadcast_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        broadcast_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        while True:
            message = {
                "cmd": "SERVER_BROADCAST",
                "ip": SERVER_IP,
                "tcp_port": TCP_PORT,
                "timestamp": time.time()
            }
            broadcast_socket.sendto(encode_message(message), ('<broadcast>', BROADCAST_PORT))
            debug(f"Broadcasting server information: {message}")
            time.sleep(HEARTBEAT_INTERVAL)


class TCPServer(threading.Thread):
    def _init_(self):
        threading.Thread._init_(self)

    def run(self):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((SERVER_IP, TCP_PORT))
        server_socket.listen(5)
        debug("TCP Server is listening for connections...")

        while True:
            conn, addr = server_socket.accept()
            debug(f"Connected to client: {addr}")
            threading.Thread(target=self.handle_client, args=(conn,)).start()

    def handle_client(self, conn):
        global peers
        while True:
            try:
                data = conn.recv(1024)
                if not data:
                    break
                message = decode_message(data)
                debug(f"Received from client: {message}")
                if message["cmd"] == "REGISTER":
                    peer = {"uuid": message["uuid"], "ip": message["ip"], "tcp_port": message["tcp_port"]}
                    peers.append(peer)
                    debug(f"New peer registered: {peer}")
                elif message["cmd"] == "MESSAGE":
                    debug(f"Message from client: {message['content']}")
                conn.send(encode_message({"status": "OK"}))
            except Exception as e:
                debug(f"Error handling client: {e}")
                break
        conn.close()


if __name__ == "__main__":
    try:
        debug("Starting server...")
        broadcast_sender = BroadcastSender()
        broadcast_sender.start()

        tcp_server = TCPServer()
        tcp_server.start()
    except KeyboardInterrupt:
        debug("Server shutting down...")

[16:00:05] Starting server...
[16:00:05] Broadcasting server information: {'cmd': 'SERVER_BROADCAST', 'ip': '192.168.233.79', 'tcp_port': 59074, 'timestamp': 1737644405.090321}
[16:00:05] TCP Server is listening for connections...
